# RKNN Conversion Guide

----------------------------

### Before you start

Before you run the scripts/python notebook from this project, it's recommended you create a separate [python virtual environment](https://docs.python.org/3/library/venv.html) so that packages installed for the conversion process don't conflict with other packages you may already have installed.


### Preinstallation (for Google Colab users)

This notebook requires the use of external python scripts, please run this snippet with the URL (`GITHUB_URL`) to the Photonvision repo, if not provided already.

In [ ]:
GITHUB_URL = "https://github.com/PhotonVision/photonvision"
SCRIPTS_FOLDER = "/scripts/rknn-convert-tool"

import sys
from urllib.parse import urlparse
import subprocess

parsed = urlparse(GITHUB_URL)
        
if parsed.netloc.lower() not in ['github.com', 'www.github.com']:
   print("URL must be GitHub URL!")
   sys.exit(1)
            
path_parts = parsed.path.strip('/').split('/')
        
if len(path_parts) < 2:
   print("Invalid Github URL! Must have org and repo in url")
   sys.exit(1)
            
org_user = path_parts[0]
repo = path_parts[1]
        
if not org_user or not repo:
   print("Invalid Github URL! Must have org and repo in url")
   sys.exit(1)
            
repo_sub_url = f"{org_user}/{repo}"
folder_url = SCRIPTS_FOLDER.strip("/")

create_onnx_raw_url = f"https://raw.githubusercontent.com/{repo_sub_url}/refs/heads/main/{folder_url}/create_onnx.py"
auto_install_script_raw_url = f"https://raw.githubusercontent.com/{repo_sub_url}/refs/heads/main/{folder_url}/autoinstallrknnapi.py"
create_rknn_raw_url = f"https://raw.githubusercontent.com/{repo_sub_url}/refs/heads/main/{folder_url}/create_rknn.py"

scriptUrls = [create_onnx_raw_url, auto_install_script_raw_url, create_rknn_raw_url]

for scriptUrl in scriptUrls:
    try:
        subprocess.run(["wget", scriptUrl]).check_returncode()
    except subprocess.CalledProcessError as e:
        print(f"Failed to run script download for url {scriptUrl}")
        print(e.output)


### Step 1: Convert to ONNX 

To convert to ONNX, simply run the `create_onnx.py` script with your model weights, see below

#### *Notice for Colab users*

Google Colab comes with an incompatible version of Numpy installed. To fix this, please run the following cells below and **restart your session** when prompted.

In [ ]:
%pip uninstall numpy -y
%pip install "numpy>=1.23.0,<2.0.0"

Then, simply run the `create_onnx.py` script to convert your `.pt` weights

In [ ]:
# where version is either yolov5, yolov8, or yolov11, and model_path is the path to your weights file (.pt)
%run create_onnx.py --version yolov8 --model_path weights.pt

### Step 2: Download RKNN API
You can either utilize a script to autodetect and install the correct Python library for you, or manually install it

#### Automatic installation
Simply run the `autoinstallrknnapi.py` script.

In [ ]:
%run autoinstallrknnapi.py

#### Manual installation
##### How to find the correct link
Go to https://github.com/airockchip/rknn-toolkit2, and click on `rknn-toolkit2`, then `packages`.
If you are running an x86_64 CPU (e.g. most Intel and AMD CPUs) click on that, otherwise choose arm64 for ARM-based computers (e.g. M-series Macs or Snapdragon processors). If you aren't sure what CPU you are running, look up your processor architecture information from system settings.

Once you have the correct CPU, you will see multiple packages. The file names will look something like `rknn_toolkit2-2.3.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl` for example. The numbers after CP correspond to your python version. If you have a Python version 3.10, for example, you want to download a package with cp310 in the name. For 3.8, you'd look for cp38, for 3.7 cp37, and so on.

Then, once you find your desired package, locate the "Raw" download button, and download the package (.whl) once you do, run pip install, replacing `rknn_toolkit2.whl` with the path to the wheel file you just downloaded

In [ ]:
%pip install rknn_toolkit2.whl

### Step 3: Convert to RKNN

Simply run the `create_rknn.py` script, replacing the arguments with your own.

#### Overview of `create_rknn.py` script

##### RKNN Conversion Script Overview

This script converts a YOLO ONNX model to RKNN format using a set of calibration images. It's designed to work with either:

- A flat directory of images (e.g. `train/images`), **or**
- A dataset directory containing a `data.yaml` file that defines `train`, `val`, and/or `test` folders.

You can use it from the command line or from inside a Python environment like this notebook.

##### Arguments

| Argument | Type | Description |
|----------|------|-------------|
| `--img_dir` (`-d`) | `str` (required) | Path to your image directory. This can either be a folder of images **or** a dataset folder with a `data.yaml`. |
| `--model_path` (`-m`) | `str` (required) | Path to your YOLO ONNX model, created in Step 1. |
| `--num_imgs` (`-ni`) | `int` (default: `300`) | Number of images to use for quantization calibration. |
| `--disable_quantize` (`-dq`) | `bool` (default: `False`) | Set to `True` to skip quantization entirely, not recommended for performance. |
| `--rknn_output` (`-o`) | `str` (default: `out.rknn`) | File path where the final RKNN model should be saved. |
| `--img_dataset_txt` (`-ds`) | `str` (default: `imgs.txt`) | File path to store the list of images used during quantization. |
| `--verbose` (`-vb`) | `bool` (default: `False`) | Enable detailed logging from RKNN during conversion. |
---


##### Notes

As this is meant to be used with [PhotonVision](https://photonvision.org) this script only allows the target platform to be RK3588 (found in Orange Pi 5 models), but feel free to modify the script to suit your needs

### Quantization Note

When performing quantization, it is critical to provide representative images of the objects or scenes you are trying to detect. These images are used to calibrate the model’s internal activations and greatly influence the final performance.

It is recommended to use 300–500 representative images that reflect the real-world input your model will encounter. As the old saying goes, *quality* over quantity.

Quantization will cause some loss in model accuracy. However, if your calibration images are chosen wisely, this accuracy drop should be minimal and acceptable. If the sampled images are too uniform or unrelated, your quantized model's performance may worsen significantly.

The script will automatically sample representative images randomly from the provided dataset. While this usually works well, please verify that the dataset contains diverse and relevant examples of your target objects. As a reminder, the images used to quantize the model are stored in the text file specified by `--img_dataset_txt`.


### Optional: Download a dataset from Roboflow for quantization

Please run the below code to download a dataset from roboflow if you do not have an images to use for quantization. Feel free to replace the link in quotes with a link to your own dataset.

In [ ]:
%wget -O roboflow.zip "https://universe.roboflow.com/ds/FaF3HbDmF7?key=iMoJR25O9H" && unzip roboflow.zip -d datasets && rm roboflow.zip

Once you have your dataset prepared, run the below script to quantize and convert your generated ONNX model from Step 1.

In [ ]:
%run create_rknn.py --img_dir ./datasets --model_path weights.onnx

And that's it! You should have an RKNN model file ready to deploy on an Orange PI